In [ ]:
import folium
import geopandas as gpd
from IPython.display import display, Image
import requests
import warnings
warnings.filterwarnings('ignore')

# ATL08 v003 Entwine Point Tiles (EPT)

This notebook demonstrates using the ATL08 IceSAT-2 [Entwine Point Tile (EPT)](https://entwine.io/entwine-point-tile.html) store.

### What are Entwine Point Tiles?

Entwine Point Tiles are a cloud-optimized octree data format for storing and visualizing massive point clouds efficiently. This format is gaining a lot of momentum and interest with an active development community.


### Visualizing with potree

Potree is a tool for visualizing EPT stores directly.

[potree.entwine.io: ATL08](https://potree.entwine.io/data/view.html?r=%22https://cumulus-map-internal.s3.amazonaws.com/file-staging/nasa-map/ATL08_ARD-beta___001/global/ept%22)

In [ ]:
img_src = "potree-global.gif"
Image(url = img_src)

### Visualizing with the 3D Tiles Service (OGC)

Cesium is a 3D tool which can be used to visualize point clouds alongside 2D data using a 3D Tiles Service.

* [3D Tiles API Endpoint](https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)
* [Demo using Cesium](http://cesium.entwine.io/?url=https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)

In [ ]:
img_src = "cesium-global.gif"
Image(url = img_src)

## Querying ATL08 EPT using the OGC Features API

We can use the features service for some basic querying of the EPT Store.

### Query By Bounding Box

In [ ]:
# Format a request to the API
api_url = "https://obnrh8ozt0.execute-api.us-east-2.amazonaws.com/collections/Global/items"

# bbox should be defined as xmin, xmax, (min value z), ymin, ymax, (max value z)
# Make a request for a bounding box over Peru
bbox="-77,-26,300,-73,0,500"

payload = {
    "f": "json",
    "limit": 100,
    "bbox": bbox,
}

r = requests.get(api_url, params = payload)

In [ ]:
# Get the results directly into a Geo Data Frame (saving to file not required but recommended)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

In [ ]:
m = folium.Map(
    location=[adf.centroid[0].y, adf.centroid[0].x],
    zoom_start=10,
    tiles='Stamen Terrain'
)


folium.GeoJson(
    adf,
    name = "geojson"
).add_to(m)

m

### Query by Granule Id

In [ ]:
granule_id = 'ATL08_20181014035224_02370107_003_01'
payload = {
    "f": "json",
    "origin": granule_id,
}

r = requests.get(api_url, params = payload)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

## More query options with pdal

The [documentation](https://maap-project.readthedocs.io/en/latest/query/testing-ept-stores.html#PDAL-Pipelines) provides additional options for how to query with PDAL (Point Cloud Data Abstraction Library).

# Thank You!

**Big thanks to** Aaron Kaulfus, Alex Mandel, Chuck Daniels, David Bitner, Dai Hai Ton That, Kaylin Bugbee, Slesa Adhikari, Sam Ayers and Seth Vincent with whom none of this would be possible.